In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

## Day 31 Lecture 2 Assignment

In this assignment, we will learn about the weighting and scaling with the K nearest neighbor algorithm. We will use the wine quality dataset loaded below and analyze the model generated for this dataset.

In [2]:
import warnings
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

import matplotlib.pyplot as plt

%matplotlib inline

<IPython.core.display.Javascript object>

In [3]:
def print_vif(x):
    """Utility for checking multicollinearity assumption
    
    :param x: input features to check using VIF. This is assumed to be a pandas.DataFrame
    :return: nothing is returned the VIFs are printed as a pandas series
    """
    # Silence numpy FutureWarning about .ptp
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        x = sm.add_constant(x)

    vifs = []
    for i in range(x.shape[1]):
        vif = variance_inflation_factor(x.values, i)
        vifs.append(vif)

    print("VIF results\n-------------------------------")
    print(pd.Series(vifs, index=x.columns))
    print("-------------------------------\n")

<IPython.core.display.Javascript object>

In [4]:
wine = pd.read_csv(
    "https://raw.githubusercontent.com/Thinkful-Ed/data-science-lectures/master/wineQualityReds.csv",
    index_col=0,
)
wine.head()

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
1,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
2,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
3,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
4,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
5,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


<IPython.core.display.Javascript object>

In [5]:
wine.shape

(1599, 12)

<IPython.core.display.Javascript object>

In [6]:
wine.dtypes

fixed.acidity           float64
volatile.acidity        float64
citric.acid             float64
residual.sugar          float64
chlorides               float64
free.sulfur.dioxide     float64
total.sulfur.dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
quality                   int64
dtype: object

<IPython.core.display.Javascript object>

Recall that we need to check for missing data.

In [7]:
wine.isna().sum()

fixed.acidity           0
volatile.acidity        0
citric.acid             0
residual.sugar          0
chlorides               0
free.sulfur.dioxide     0
total.sulfur.dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

<IPython.core.display.Javascript object>

Convert quality to a binary variable, with the dividing line between 5 and 6.

In [8]:
wine["quality"] = wine["quality"] > 5
wine["quality"] = wine["quality"].astype(int)
wine.head()

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
1,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
2,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
3,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
4,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
5,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


<IPython.core.display.Javascript object>

Create a train test split with 20% of the data in the test subsample.

In [9]:
print_vif(wine.drop(columns=["quality"]))

VIF results
-------------------------------
const                   1.710538e+06
fixed.acidity           7.767512e+00
volatile.acidity        1.789390e+00
citric.acid             3.128022e+00
residual.sugar          1.702588e+00
chlorides               1.481932e+00
free.sulfur.dioxide     1.963019e+00
total.sulfur.dioxide    2.186813e+00
density                 6.343760e+00
pH                      3.329732e+00
sulphates               1.429434e+00
alcohol                 3.031160e+00
dtype: float64
-------------------------------



<IPython.core.display.Javascript object>

In [10]:
# addition column
X = wine.drop(columns=["quality", "fixed.acidity"])
y = wine["quality"]

<IPython.core.display.Javascript object>

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

<IPython.core.display.Javascript object>

Create a KNN model with k=5 and report the accuracy scores. Then make a second model using the scaled data and compare your results.

In [12]:
model = KNeighborsClassifier(5)
model.fit(X_train, y_train)

model.score(X_train, y_train)

0.7928068803752932

<IPython.core.display.Javascript object>

In [13]:
model.score(X_test, y_test)

0.60625

<IPython.core.display.Javascript object>

Scale only the independent variables using the minmax scaler.

In [14]:
scaler = MinMaxScaler()
scaler = StandardScaler()

<IPython.core.display.Javascript object>

In [15]:
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

<IPython.core.display.Javascript object>

In [16]:
model = KNeighborsClassifier(5)
model.fit(X_train, y_train)

model.score(X_train, y_train)

0.8240813135261923

<IPython.core.display.Javascript object>

In [17]:
model.score(X_test, y_test)

0.74375

<IPython.core.display.Javascript object>

When generating a KNN model, we can use the weighted model by setting `weights='distance'`. We can also write our own custom weights function.

Write a custom weight function that assigns the weight of 1/sqrt(distance) and use this function in your model. Report the accuracy score.

Hint: Use the `_get_weights` function in scikit learn as a resource. The code is <a href="https://github.com/scikit-learn/scikit-learn/blob/fdbaa58acbead5a254f2e6d597dc1ab3b947f4c6/sklearn/neighbors/base.py#L63" title="_get_weights">here</a>.

In [18]:
# Not actually implementing the function yet
# I want to see what data is being passed into it
def print_input(dists):
    print(dists)
    return dists

<IPython.core.display.Javascript object>

In [19]:
model = KNeighborsClassifier(weights=print_input)
model.fit(X_train, y_train)
model.predict(X_train[[0], :])

[[0.         0.         1.32019079 1.32019079 1.73983086]]


array([1])

<IPython.core.display.Javascript object>

In [20]:
example_input = np.array([[0.0, 0.0, 1.32019079, 1.32019079, 1.73983086]])

# we have division by zero
# if we do have 0 distance we probably do want to give it a lot of weight
# infinite weight is a lot, but maybe you go with it
1 / np.sqrt(example_input)

<ipython-input-20-32e40c937974>:6: RuntimeWarning: divide by zero encountered in true_divide
  1 / np.sqrt(example_input)


array([[       inf,        inf, 0.87032538, 0.87032538, 0.75813489]])

<IPython.core.display.Javascript object>

In [21]:
def custom_weight(dists):
    # Silence warnings about divide by zero
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        weighted_dists = 1 / np.sqrt(dists)

    # restricting weighted dists to between 0 and 2
    return np.clip(weighted_dists, 0, 2)

<IPython.core.display.Javascript object>

In [22]:
model = KNeighborsClassifier(weights=custom_weight)
model.fit(X_train, y_train)

model.score(X_train, y_train)

0.9132134480062549

<IPython.core.display.Javascript object>

In [23]:
model.score(X_test, y_test)

0.765625

<IPython.core.display.Javascript object>

Finding informed values for the hyperparameters

In [24]:
grid = {"n_neighbors": range(1, 15), "weights": ["uniform", "distance", custom_weight]}

model_cv = GridSearchCV(KNeighborsClassifier(), grid)
model_cv.fit(X_train, y_train)

model_cv.best_params_

{'n_neighbors': 11, 'weights': 'distance'}

<IPython.core.display.Javascript object>

In [25]:
model_cv.score(X_train, y_train)

1.0

<IPython.core.display.Javascript object>

In [26]:
model_cv.score(X_test, y_test)

0.8

<IPython.core.display.Javascript object>